In [1]:
import pandas as pd
import numpy as np 

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",inplace=True, axis=1)

In [3]:
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


* 데이터의 Entropy 구하기

In [4]:
def get_info(df):
    y_labels = df['class_buys_computer'].unique()
    pi_probas = np.array([len(df[df['class_buys_computer'] == attribute]) \
                                    / len(df) for attribute in y_labels])
    calculate_log = np.log2(pi_probas[pi_probas != 0])
    return -np.sum(pi_probas[pi_probas != 0] * calculate_log)
get_info(pd_data)

0.94028595867063114

* Feature로 분류 했을 때 정보량 구하기

In [5]:
def feature_info(df, feature_name):
    attributes = df[feature_name].unique()
    attribute_probas = np.array([len(df[df[feature_name] == j]) / len(df) for j in attributes])
    splits_attribute_df_entropy = np.array([get_info(df[df[feature_name]==j]) for j in attributes])
    #print(attribute_probas, splits_attribute_df_entropy)
    result = np.sum(attribute_probas * splits_attribute_df_entropy)
    return result

In [6]:
print('전체 데이터를 age로 분류 했을 때 정보량 :', feature_info(pd_data, 'age'))
print('전체 데이터를 income로 분류 했을 때 정보량 :', feature_info(pd_data, 'income'))
print('전체 데이터를 student로 분류 했을 때 정보량 :', feature_info(pd_data, 'student'))
print('전체 데이터를 credit_rating로 분류 했을 때 정보량 :', feature_info(pd_data, 'credit_rating'))

전체 데이터를 age로 분류 했을 때 정보량 : 0.693536138896
전체 데이터를 income로 분류 했을 때 정보량 : 0.911063393012
전체 데이터를 student로 분류 했을 때 정보량 : 0.788450457308
전체 데이터를 credit_rating로 분류 했을 때 정보량 : 0.892158928262


* Information gain 값 구하기
    * IG값은 커야 좋음. -> 얻는 정보량이 많아짐.
    * using_data = [IG값 리스트, 가장 IG값이 큰 feature name, IG값을 tuple 형식으로 표현] 

In [9]:
def pick_feature(df, attributes_list):
    
    data_info = get_info(df)
    attr_infos = np.array([feature_info(df, attr_name) for attr_name in attributes_list])
    information_gain_list = np.array([data_info - attr_info for attr_info in attr_infos])    
    info_dict = {attr_name : info_gain for attr_name, info_gain in zip(attributes_list, information_gain_list)}
    #print(info_dict)
    spliting_information = sorted(info_dict.items(), key=lambda x: x[1], reverse=True)[0]
    using_data = [information_gain_list, spliting_information]
    
    return using_data
pick_feature(pd_data, pd_data.columns[:-1])

{'income': 0.029222565658954869, 'age': 0.24674981977443933, 'credit_rating': 0.048127030408269489, 'student': 0.15183550136234159}


[array([ 0.24674982,  0.02922257,  0.1518355 ,  0.04812703]),
 ('age', 0.24674981977443933)]

* id3 알고리즘 : recursive하게 만들어야 하는데... 좀 더 고민을 해봐야 할 것 같습니다. 그래서 일단은 recursive하지 않게 만들어 보았습니다. 
    * 코드를 짜고 보니까 너무 이 데이터에 최적화 된 것 같습니다.

In [8]:
def id3_algorithms(df):
    total_save = {} 
    sub_save = {}
    column_names = df.columns[:-1]
    result = pick_feature(df, column_names) # 처음 spliting_node 정하기
    spliting_feature = result[1][0] 
    ''' (result[0]은 information gain 값, 
         result[1]은 tuple형식으로 ('spliting_feature_name', information gain 값)) 
         그래서 result[1][0]'''
    attributes_list = df[spliting_feature].unique() 
    # age의 경우 세가지 속성 가지고 있음. 이 경우 [youth, middle_aged, senior]
    total_save[spliting_feature] = '' 
    # key 값 age 초기화
    for attribute in attributes_list:
    # age의 경우 youth, middle_aged, senior 이 세가지 attribute가 attribute_list안에 있음
        split_df = df.loc[df[spliting_feature] == attribute] # age에서 가지 뻗기, 데이터 프레임 분할
        feature_attrs = split_df.columns[split_df.columns != spliting_feature][:-1] 
        #target 제외, 처음에 분할된 node : age 제외
        for attr in feature_attrs: #feature_attrs = [income, student, credit_rating]
            print(attribute, attr, feature_info(split_df, attr))
            if feature_info(split_df, attr) == 0: 
                #Spliting node 찾기. 분할된 데이터에서 attr로 분류했을 때 정보량이 0인 경우
                if get_info(split_df[split_df[spliting_feature] == attribute]) == 0:
                    '''age node에서 middle_aged node로 뻗어 나왔을 때 
                       middle_aged의 경우 이미 구분이 되어 있으므로 entroy 값 : 0''' 
                    sub_save[attribute] = [{split_df[split_df.columns[-1]].unique()[0]
                                            : np.array(split_df.index)}]
                    '''middle_aged의 경우 가지를 뻗을 필요가 없음 
                       그래서 바로 [{yes일 때 : index}]로 표현'''
                else:
                    # age에서 뻗어 나왔을 때 각 노드별(youth node, senior node) 정보량 값이 0이 아닌 경우
                    sub_feature_attributes = split_df[attr].unique() 
                    ''' 여기서 attr : 위에서 feature_info(split_df, attr) == 0일 때 attr,
                        attribute가 youth인 경우 student를 의미, youth node에서 student 속성이 spliting node가 됨 
                        attr이 student인 경우 sub_feature_attributes에는 student의 속성인 no, yes 포함'''
                    sub_save_2 = {attr : ''} #sub_save_2 = key값으로 attr, value 값 : ''로 초기화
                    save_list = []
                    for sub_attr in sub_feature_attributes:
                        sub_split_df = split_df[split_df[attr] == sub_attr] 
                        '''attribute가 youth일 때(=split_df) attr : student인 경우 
                           sub_attr = no, yes인 경우 데이터프레임 분할'''
                        save_list.append({sub_split_df[sub_split_df.columns[-1]].unique()[0]
                                            : np.array(sub_split_df.index)})
                        '''dict형태로 yes일 때  index, 
                           no일 때 index 각각 key,value 값으로 표현 
                           dict를 save_list에 append'''
                    sub_save_2[attr] = save_list 
                    '''youth인 경우 attr : student.  
                       {student : [{no : no일 때 index, yes : yes일 때 index}]}'''
                    sub_save[attribute] = sub_save_2 
                    ''' {youth : {student : [{no : no일 때 index}, {yes : yes일 때 index}]}}'''
                        
    total_save[spliting_feature] = sub_save
    return total_save
id3_algorithms(pd_data)

youth income 0.4
youth student 0.0
youth credit_rating 0.950977500433
middle_aged income 0.0
middle_aged student 0.0
middle_aged credit_rating 0.0
senior income 0.950977500433
senior student 0.950977500433
senior credit_rating 0.0


{'age': {'middle_aged': [{'yes': array([ 2,  6, 11, 12], dtype=int64)}],
  'senior': {'credit_rating': [{'yes': array([3, 4, 9], dtype=int64)},
    {'no': array([ 5, 13], dtype=int64)}]},
  'youth': {'student': [{'no': array([0, 1, 7], dtype=int64)},
    {'yes': array([ 8, 10], dtype=int64)}]}}}